# Numerical Methods 1
### [Gerard Gorman](http://www.imperial.ac.uk/people/g.gorman), [Matthew Piggott](http://www.imperial.ac.uk/people/m.d.piggott), [Christian Jacobs](http://www.christianjacobs.uk)

# Lecture ?: Numerical Linear Algebra I

## Learning objectives:

* Manipulation of matrices and matrix equations in Python.
* Reminder on properties of matrices (from MM1): determinants, singularity etc.
* Algorithms for the solution of linear systems (direct and indirect/iterative).
* Gaussian elimination and LU factorisation.

## Introduction - Linear (matrix) systems

Recall from your Mathematical Methods I course that the we can re-write a system of simultaneous (linear) equations in matrix form.  For example, in week 4 of MM1 you considered the following example:

\begin{eqnarray*}
  2x + 3y &=& 7 \\
   x - 4y &=& 3
\end{eqnarray*} 

and it was noted that this can be written in matrix form as 

$$
\left(
  \begin{array}{rr}
    2 & 3 \\
    1 & -4  \\
  \end{array}
\right)\left(
  \begin{array}{c}
    x \\
    y \\
  \end{array}
\right) = \left(
  \begin{array}{c}
    7 \\
    3 \\
  \end{array}
\right)
$$

More generally, consider the arbitrary system of $m$ linear equations for $m$ unknowns

\begin{eqnarray*}
  a_{11}x_1 + a_{12}x_2 + \dots + a_{1m}x_m &=& b_1 \\ 
  a_{21}x_1 + a_{22}x_2 + \dots + a_{2m}x_m &=& b_2 \\ 
  \vdots &=& \vdots \\ 
  a_{m1}x_1 + a_{m2}x_2 + \dots + a_{mm}x_m &=& b_m
\end{eqnarray*}

where $a_{ij}$ are the constant coefficients, $x_j$ are the unknown variables, and $b_i$
are the terms on the right hand side.  Here the index $i$ is referring to the equation number
(the row in the matrix below), with the index $j$ referring to the component of the unknown
vector $\pmb{x}$ (the column of the matrix).

This system of equations can be represented as the matrix equation $A\pmb{x}=\pmb{b}$: 

$$
\left(
  \begin{array}{cccc}
    a_{11} & a_{12} & \dots & a_{1m} \\
    a_{21} & a_{22} & \dots & a_{2m} \\
    \vdots & \vdots & \ddots & \vdots \\
    a_{m1} & a_{m2} & \dots & a_{mm} \\
  \end{array}
\right)\left(
         \begin{array}{c}
           x_1 \\
           x_2 \\
           \vdots \\
           x_m \\
         \end{array}
       \right)  = \left(
                   \begin{array}{c}
                     b_1 \\
                     b_2 \\
                     \vdots \\
                     b_m \\
                   \end{array}
                 \right)
$$


For the above $2 \times 2$ example of two equations in two unknowns in MM1 you substitution (multiply the second equation by 2 and subtract it from the first to find y, and then compute x) to easily give the solution $x=37/11$, $y=1/11$.

In MM1 you also considered $3 \times 3$ examples which were a little more complicated but still doable.  This lecture considers the case of $N \times N$ where $N$ could easily by billions! This case arises when you solve a differential equation numerically on a discrete mesh or grid. Here you would typically obtain one unknown and one (discrete, linear or nonlinear) equation at very grid point. You could generate an arbitrarily large matrix system simply by generating a finer and finer mesh.

Note that you will solve differential equations numerically in the follow-up course Numerical Methods II.

Cases where the matrix is non-square, i.e. of shape $m \times n$ where $m\ne n$ correspond to the (over- or under-determined) system where you have more or less equations than unknowns - we won't consider these in this lecture. 


## Matrices in Python

We have already used numpy arrays to store one-dimensional vectors of numbers.

The convention is that these are generally considered to be *column* vectors and have shape $1 \times N$.

We can extend to higher dimensions through the more general concept of tensors: a vector can be
represented as a one-dimensional array and is a 1st-order tensor;  a matrix is a two-dimensional array and us a 2nd-order tensor; and so on.

Note that this is the total number of indices required to select each component of the array, i.e. we can identify each component of the vector $\pmb{v}$ by $v_i$, and each component of the vector $A$ by
$A_{ij}$.  

Note that it is a convention that vectors are either underlined or bold, and generally lower case letters, whereas matrices are plain capital letters.

Note that the terms *dimension*, *order*, *degree* and *rank* can all be used for this quantity, but beware that the word rank also has a different usage in the context of matrices and tensors!

Here is an example of how we can extend our use of the numpy array object to two dimensions in order to define a matrix $A$ and some examples of some operations we can make on it.

In [60]:
import numpy
from scipy import linalg
A=numpy.array([[10., 2., 1.],[6., 5., 4.],[1., 4., 7.]])
print(A)
print(numpy.size(A))    # the total size of the array storing A - here 9 for a 3x3 matrix
print(numpy.ndim(A))  # the dimension of the matrix A
print(numpy.shape(A))  # the shape of the matrix A
print(A.T)            # the transpose of the matrix A
print(linalg.inv(A))  # the inverse of the matrix A - computed using a scipy algorithm
print(linalg.det(A))  # the determinant of the matrix A - computed using a scipy algorithm
print(numpy.dot(A,linalg.inv(A)))  # multiply A with its inverse using numpy's dot
print(A.dot(linalg.inv(A)))  # same way to achieve the same thing
print(A*linalg.inv(A))     # note that the * operator simply does operations element-wise

print(numpy.zeros(3))  # how to initialise a vector of zeros 
print(numpy.zeros((3,3))) # how to initialise a matrix of zeros 
print(numpy.zeros((3,3,3)))  # how to initialise a 3rd-order tensor of zeros

print(numpy.eye(3))  # how to initialise the identity matrix, I or Id

[[ 10.   2.   1.]
 [  6.   5.   4.]
 [  1.   4.   7.]]
9
2
(3, 3)
[[ 10.   6.   1.]
 [  2.   5.   4.]
 [  1.   4.   7.]]
[[ 0.14285714 -0.07518797  0.02255639]
 [-0.28571429  0.51879699 -0.2556391 ]
 [ 0.14285714 -0.28571429  0.28571429]]
133.00000000000003
[[  1.00000000e+00   1.11022302e-16  -5.55111512e-17]
 [ -4.44089210e-16   1.00000000e+00   0.00000000e+00]
 [ -4.44089210e-16   6.66133815e-16   1.00000000e+00]]
[[  1.00000000e+00   1.11022302e-16  -5.55111512e-17]
 [ -4.44089210e-16   1.00000000e+00   0.00000000e+00]
 [ -4.44089210e-16   6.66133815e-16   1.00000000e+00]]
[[ 1.42857143 -0.15037594  0.02255639]
 [-1.71428571  2.59398496 -1.02255639]
 [ 0.14285714 -1.14285714  2.        ]]
[ 0.  0.  0.]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
[[[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]]
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]


Let's quickly consider the $2 \times 2$ case from MM1 recreated above where we claimed that $x=37/11$, $y=1/11$.  

To solve the matrix equation 

$$ A\pmb{x}=\pmb{b} $$

we can simply multiply both sides by the inverse of the matrix $A$ (if $A$ is invertible and if we know what the inverse is of course!):

\begin{align}
A\pmb{x} & = \pmb{b}\\
\implies A^{-1}A\pmb{x} & = A^{-1}\pmb{b}\\
\implies I\pmb{x} & = A^{-1}\pmb{b}\\
\implies \pmb{x} & = A^{-1}\pmb{b}
\end{align}

so we can find the solution $\pmb{x}$ by multiplying the inverse of $A$ with the RHS vector $\pmb{b}$.

In [61]:
A=numpy.array([[2., 3.],[1., -4.]])
print(linalg.det(A))   # check first whether the determinant of A is non-zero - see below for reasons why!
b=numpy.array([7., 3.])
print(numpy.dot(linalg.inv(A),b))
print(37./11.)
print(1./11.)

-11.0
[ 3.36363636  0.09090909]
3.3636363636363638
0.09090909090909091


### <span style="color:blue">Aside: matrix objects </span>

Note that numpy does possess a matrix object as a sub-class of the numpy array.  We can cast the above two-dimensional arrays into matrix objects and then the star operator does yield the expected matrix product:

In [64]:
print(type(A))  # the is an n-dimensional array (n-2 here)
print(type(numpy.mat(A)))  # this casts the array A into the matrix class
# for these objects * is standard matrix multuiplication and we can check that A*A^{-1}=I as expected
print(numpy.mat(A)*numpy.mat(linalg.inv(A)))

<class 'numpy.ndarray'>
<class 'numpy.matrixlib.defmatrix.matrix'>
[[ 1.  0.]
 [ 0.  1.]]


### <span style="color:blue">Slicing </span>

Note that just as for arrays or lists, we can use *slicing*  in order to extract components of matrices, for example:

In [66]:
A=numpy.array([[10., 2., 1.],[6., 5., 4.],[1., 4., 7.]])
print(A)
print(A[0,1])   # single entry, first row, second column
print(A[0,:])   # first row
print(A[-1,:])  # last row
print(A[:,1])   # second column
print(A[1:3,1:3]) # extract a 2x2 sub-matrix

[[ 10.   2.   1.]
 [  6.   5.   4.]
 [  1.   4.   7.]]
2.0
[ 10.   2.   1.]
[ 1.  4.  7.]
[ 2.  5.  4.]
[[ 5.  4.]
 [ 4.  7.]]


## Properties of matrices: singularity, determinants etc

Consider $N$ linear equations in $N$ unknowns, $A\pmb{x}=\pmb{b}$. 

From MM1 you learnt that this system has a *unique solution* provided that the determinant of A, $\det(A)$, is non-zero. In this case the matrix is said to be *non-singular*.

If $\det(A)=0$ (with $A$ then termed a *singular matrix*), then the linear system does *not* have a unique solution, it may have either infinite *or* no solutions.

For example, consider

$$
\left(
  \begin{array}{rr}
    2 & 3 \\
    4 & 6  \\
  \end{array}
\right)\left(
  \begin{array}{c}
    x \\
    y \\
  \end{array}
\right) = \left(
  \begin{array}{c}
    4 \\
    8 \\
  \end{array}
\right)
$$

The second equation is simply twice the first, and hence a solution to the first equation is also automatically a solution to the second equation. 

We hence only have one *linearly-independent* equation, and our problem is under-constrained: we effectively only have one eqution for two unknowns with infinitely many possibly solutions. 

If we replaced the RHS vector with $(4,7)^T$, then the two equations would be contradictory: in this case we have no solutions.

Note that a set of vectors where one can be written as a linear sum of the others are termed *linearly-dependent*. When this is not the case the vectors are termed *linearly-independent*.

The following properties of a square $N\times N$ matrix are equivalent:

* $\det(A)\ne 0$ - A is non-singular

* The columns of $A$ are linearly independent

* The rows of $A$ are linearly independent

* The columns of A *span* $N$-dimensional space (recall MM1 - we can reach any point in $\mathbb{R}^N$ through a linear combination of these vectors - note that this is simply what the operation $A\pmb{x}$ is doing of course if you write it out)

* $A$ is invertible, i.e. there exists a matrix $A^{-1}$ such that $A^{-1}A = A A^{-1}=I$

* the matrix system $A\pmb{x}=\pmb{b}$ has a unique solution for every vector $b$


### Ill-conditioning

### Direct vs iterative methods

Two types of methods exist to solve matrix systems.  These are termed direct methods and iterative (or indirect) methods.

Direct methods perform operations on the linear equations (the matrix system), e.g. the substitution we saw above for the MM1 example, transforming the equations into equivalent ones with the aim of eliminating unknowns from some of the equations and hence allowing for easy solution through back (or forward) substitution.

In MM1 you learnt Cramer's rule which gives an explicit formula for the inverse of a matrix, or for the solution of a linear matric system.  It was pointed out that the cost (or complexity) scaled like $(N+1)!$, whereas the Gaussian elimination (which is basically the susbtitution method done above) scaled like $N^3$.  For large $N$ Gaussian elimination will clearly be more efficient - you considered the case where $N=100$ in MM1 for example.

However, as pointed out above $N$ could be billions for hard-core applications such as in weather forecasting. In this case the $N^3$ operations required of a direct algorithm such as Guassian elimination is also prohibitive. 

In order to reduce this cost, ideally to a level that is linear in $N$, *iterative* algorithms were devised. These start with a guess at the solution ($\pmb{x}_0$), they calculate the residual ($A\pmb{x}_0 - \pmb{b}$) which will obviously not be zero unless you were very lucky with your initial guess, and then *iteratively* seek to improve on this solution to drive down the residual.  This iteration will stop at some small (non-zero) residual tolerance level, yielding an approximation to the solution, but not the exact solution we would obtain with direct methods.  The residual tolerance stopping criteria therefore needs to be thought about carefully, e.g. depending on how accurate a solution $\pmb{x}$ we require.

We will consider iterative methods next week. First we consider the direct method Gaussian elimination.


## Gaussian elimination




## LU factorisation

## Matrix inversion

### <span style="color:blue">Example: ??? </span>

